# 자연어_CH3
## 자연어 처리 개요

### 텍스트 유사도

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

sent = ('휴일인 오늘도 서쪽을 중심으로 폭염이 이어졌는데요, 내일은 반가운 비 소식이 있습니다.', 
        '폭염을 피해서 휴일에 놀러왔다가 갑작스런 비로 인해 망연자실하고 있습니다.')

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sent) # 문장 벡터화 진행

idf = tfidf_vectorizer.idf_
print(dict(zip(tfidf_vectorizer.get_feature_names(), idf))) # 각 수치에 대한 값 시각화

# TF-IDF로 벡터화한 값

{'갑작스런': 1.4054651081081644, '내일은': 1.4054651081081644, '놀러왔다가': 1.4054651081081644, '망연자실하고': 1.4054651081081644, '반가운': 1.4054651081081644, '비로': 1.4054651081081644, '서쪽을': 1.4054651081081644, '소식이': 1.4054651081081644, '오늘도': 1.4054651081081644, '이어졌는데요': 1.4054651081081644, '인해': 1.4054651081081644, '있습니다': 1.0, '중심으로': 1.4054651081081644, '폭염을': 1.4054651081081644, '폭염이': 1.4054651081081644, '피해서': 1.4054651081081644, '휴일에': 1.4054651081081644, '휴일인': 1.4054651081081644}


### 자카드 유사도
- 두 문장을 각각 단어의 집합으로 만든 뒤 두 집합을 통해 유사도를 측정하는 방식 중 하나
- 유사도를 측정하는 방법은 두 집합의 교집합인 공통된 단어의 개수를 두 집합의 합집합, 즉 전체 단어의 수로 나누면 된다.
- 1에 가까울수록 유사도가 높다는 의미

### 코사인 유사도
- 코사인 유사도는 두 개의 벡터값에서 코사인 각도를 구하는 방법
- 값은 -1과 1 사이의 값을 가지고 1에 가까울수록 유사하다는 것을 의미
- 두 벡터 간의 각도를 구하는 것이기 때문에 방향성의 개념이 더해진다.

In [2]:
# 코사인 유사도
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2]) # 첫 번째와 두 번째 문장 비교

array([[0.05629716]])

### 유클리디언 유사도
- 가장 기본적인 거리를 측정하는 유사도 공식
- 유클리디언 거리, L2 거리 라고 불린다.
- n 차원 공간에서 두 점 사이의 최단 거리를 구하는 접근법
- 1 이상의 값을 가질 수 있다.

In [3]:
from sklearn.metrics.pairwise import euclidean_distances

euclidean_distances(tfidf_matrix[0:1], tfidf_matrix[1:2])

array([[1.37382884]])

In [4]:
# L1 정규화 방법 (L1 - Normalization)
# 각 벡터 안의 요소 값을 모두 더한 것이 크기가 1이 되도록 벡터들의 크기를 조절하는 방법
# 벡터의 모든 값을 더한 후 이 값으로 각 벡터의 값을 나누면 된다.
"""
제한이 없는 유사도 값은 사용하기 어렵기 때문에 값을 제한해야 한다. (0 과 1 사이의 값을 갖도록 만든다.)
앞서 각 문장을 벡터화했었는데, 이 벡터를 일반화(Normalize)한 후 다시 유클리디언 유사도를 측정하면 0과 1 사이의 값을 갖게 된다.
"""

import numpy as np

def l1_normalize(v):
    norm = np.sum(v)
    return v/norm

tfidf_norm_l1 = l1_normalize(tfidf_matrix)
euclidean_distances(tfidf_norm_l1[0:1], tfidf_norm_l1[1:2])

array([[0.22387021]])

### 맨하탄 유사도
- 맨하탄 거리를 통해 유사도를 측정하는 방법
- 맨하탄 거리란 사각형 격자로 이뤄진 지도에서 출발점에서 도착점까지를 가로지르지 않고 갈 수 있는 최단거리를 구하는 공식
- 맨하탄은 L1 거리라고 부른다
- 값이 계속 커질 수 있기 때문에 L1 정규화 방법을 사용해 벡터 안의 요소 값을 정규화한 뒤 유사도를 측정한다.

In [5]:
from sklearn.metrics.pairwise import manhattan_distances

manhattan_distances(tfidf_norm_l1[0:1], tfidf_norm_l1[1:2])

array([[0.92479112]])

In [ ]:
import os
import re

import pandas as pd
import tensorflow as tf
from tensorflow.keras import utils

data_set = tf.keras.utils.get_file(fname = 'imdb.tar.gz',\
                                  origin='http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz',\
                                  extract = True) # 압축 해제

In [ ]:
# 압축이 풀린 데이터들이 directory 안에 txt 파일 형태로 있어서 판다스의 데이터프레임을 만들기 위해서는 변환 작업을 진행해야 한다.

# 각 파일에서 리뷰 텍스트를 불러오는 함수
def directory_data(directory): # 데이터를 가져올 Directory를 인자로 받는다.
    data = {}
    data['review'] = []
    for file_path in os.listdir(directory):
        with open(os.path.join(directory, file_path), "r", encoding='UTF-8')as file:
            data["review"].append(file.read())
            
    return pd.DataFrame.from_dict(data)


# 각 리뷰에 해당하는 라벨값을 가져오는 함수
def data(directory):
    pos_df = directory_data(os.path.join(directory, "pos"))
    neg_df = directory_data(os.path.join(directory, "neg"))
    pos_df['sentiment'] = 1
    neg_df['sentiment'] = 0
    
    return pd.concat([pos_df, neg_df])


# 앞에서 설명한 두 함수를 호출해서 판다스 데이터프레임을 반환받는 구문
train_df = data(os.path.join(os.path.dirname(data_set), "aclImdb", "train"))
test_df = data(os.path.join(os.path.dirname(data_set), "aclImdb", "test"))

train_df.head()

In [ ]:


reviews = list(train_df['review'])

# reviews 에는 각 문장을 리스트로 담고 있다. 
# 단어를 토크나이징하고 문장마다 토크나이징된 단어의 수를 저장하고,
# 그 단어들을 붙여 알파벳의 전체 개수를 저장하는 부분을 만들어 보자

# 문자열 문장 리스트를 토크나이징
tokenized_reviews = [r.split() for r in reviews]


# 토크나이징된 리스트에 대한 각 길이를 저장
review_len_by_token = [len(t) for t in tokenized_reviews]


# 토크나이징된 것을 붙여서 음절의 길이를 저장
review_len_by_eumjeol = [len(s.replace(' ','')) for s in reviews]

"""
위와 같이 만드는 이유는 문장에 포함된 단어와 알파벳의 개수에 대한 데이터 분석을 수월하게 만들기 위해서이다.
"""


In [ ]:



# 히스토그램으로 문장을 구성하는 단어의 개수와 알파벳 개수를 알아보자

import matplotlib.pyplot as plt

# 그래프에 대한 이미지 크기 선언
# figsize: (가로, 세로) 형태의 튜플로 입력
plt.figure(figsize = (12, 5))

# 히스토그램 선언
# bins : 히스토그램 값에 대한 버킷 범위
# alpha : 그래프 색상 투명도
# color : 그래프 색상
# label: 그래프에 대한 라벨
plt.hist(review_len_by_token, bins = 50, alpha = 0.5, color='r', label='word')
plt.hist(review_len_by_eumjeol, bins = 50, alpha = 0.5, color = 'b', label = 'alphabet')
plt.yscale('log', nonposy = 'clip')

# 그래프 제목
plt.title('Review Length Histogram')

# 그래프 x 축 라벨
plt.xlabel('Review Length')

# 그래프 y 축 라벨
plt.ylabel('Number of Reviews')

plt.show()



In [ ]:
# 문장에 대한 길이 분포
# 빨간색 히스토그램은 단어 개수에 대한 히스토그램이고, 파란색은 알파벳 개수의 히스토그램이다.\

# 데이터 분포를 통계치로 수치화
import numpy as np

print('문장 최대 길이: {}'.format(np.max(review_len_by_token)))
print('문장 최소 길이: {}'.format(np.min(review_len_by_token)))
print('문장 평균 길이: {:.2f}'.format(np.mean(review_len_by_token)))
print('문장 길이 표준편차: {:.2f}'.format(np.std(review_len_by_token)))
print('문장 중간 길이: {}'.format(np.median(review_len_by_token)))

# 사분위의 대한 경우는 0~100 스케일로 돼 있음
print('제1사분위 길이: {}'.format(np.percentile(review_len_by_token, 25)))
print('제3사분위 길이: {}'.format(np.percentile(review_len_by_token, 75)))

In [ ]:
plt.figure(figsize=(12,5))
# 박스 플롯 생성
# 첫 번째 인자: 여러 분포에 대한 데이터 리스트 입력
# labels: 입력한 데이터에 대한 라벨
# showmeans: 평균값을 마크함

plt.boxplot([review_len_by_token],
             labels = ['token'],
             showmeans = True)

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

wordcloud = WordCloud(stopwords = STOPWORDS, background_color = 'black', width = 800,
                     height = 600).generate(' '.join(train_df['review']))

plt.figure(figsize = (15, 10))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

# 데이터에 포함된 단어의 등장 횟수에 따라 단어의 크기가 커지는데 'br'이 엄청 크다
# <br> 과 같은 HTML 태그가 포함돼 있기 때문인데, 제거해야 한다 이는 4장의 전처리 단계에서 배운다.

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

sentiment = train_df['sentiment'].value_counts()
fig, axe = plt.subplots(ncols=1)
fig.set_size_inches(6, 3)
sns.countplot(train_df['sentiment'])
